In [ ]:
## Goes with the buildPayloadsAndWrappers.yml file

In [ ]:
from mythic import mythic

In [ ]:
mythic_instance = await mythic.login(
        username="mythic_admin",
        password="mythic_password",
        server_ip="mythic_nginx",
        server_port=7443,
        timeout=-1
    )

In [ ]:
### trigger: manual
# This can only be triggered through the UI or via Scripting
search_for_workflow = """
query FindWorkflows($name: String!) {
  eventgroup(where: {name: {_eq: $name}, active: {_eq: true}, deleted: {_eq: false}, approved_to_run: {_eq: true}}) {
    id
  }
}
"""
workflows = await mythic.execute_custom_query(
    mythic=mythic_instance,
    query=search_for_workflow,
    variables={"name": "generate apollo payloads"}
)
print(workflows)
for workflow in workflows["eventgroup"]:
    triggered = await mythic.execute_custom_query(
        mythic=mythic_instance,
        query="""
        mutation TriggerWorkflowManually($eventgroup_id: Int!){
            eventingTriggerManual(eventgroup_id: $eventgroup_id){
                status
                error
            }
        }
        """,
        variables={"eventgroup_id": workflow["id"]}
    )
    print(triggered)

In [ ]:
### action: payload_create (normal egress payload)
#  - name: "apollo bin"
#    description: "generate shellcode"
#    action: "payload_create"
#    action_data:
#      payload_type: "apollo" <-- identify the payload type
#      description: "apollo test payload shellcode"
#      selected_os: "Windows" <-- identify the operating system version
#      build_parameters: <-- specify build parameters with name and values, an array of dictionaries
#      - name: "output_type"
#        value: "Shellcode"
#      filename: "apollo.bin"
#      c2_profiles: <-- specify c2 profiles as an array of dictionaries
#      - c2_profile: "websocket"
#        c2_profile_parameters: <-- for each profile, specify all the values you want to specify, otherwise defaults will be used
#          AESPSK: "aes256_hmac"
#          callback_host: "ws://192.168.0.118"
#          tasking_type: "Push"
#      commands: <-- identify any commands you want included
#      - shell
#      - exit
#      - load
#    outputs: <-- identify what outputs you want from here
#      PayloadUUID: "uuid" <-- because we created a payload, we have all of the payload's attriutes to pull from

In [ ]:
payload_attribute_query = """
query MyQuery {
  payload(limit: 1) {
    apitokens_id
    auto_generated
    build_container
    build_message
    build_phase
    build_stderr
    build_stdout
    callback_alert
    creation_time
    deleted
    description
    eventstepinstance_id
    file_id
    id
    operation_id
    operator_id
    os
    payload_type_id
    task_id
    timestamp
    uuid
    wrapped_payload_id
  }
}
"""
fields = await mythic.execute_custom_query(
    mythic=mythic_instance,
    query=payload_attribute_query
)
print(fields)

In [ ]:
## action: custom_function

#- name: "bin opsec checker"
#    description: "check finished payloads for bad keywords"
#    action: "custom_function"
#    depends_on:
#    - "apollo bin" <-- only gets executed after the step named "apollo bin" finishes successfully
#    inputs:
#      PayloadUUID: "apollo bin.PayloadUUID" <-- save the named output from "apollo bin"
#      APIToken: "mythic.apitoken" <-- ask Mythic for an APIToken for this step
#    action_data:
#      function_name: "opsecStrings" <-- function within container to execute
#      container_name: "fileProcessor" <-- container to task

In [ ]:
## action: conditional_check (potentially skip steps)

# - name: "conditional checker"
#    description: "check for more things"
#    action: "conditional_check"
#    depends_on:
#    - "bin opsec checker" <-- this named step must happen first
#    inputs:
#      APIToken: "mythic.apitoken" <-- ask Mythic for a temporary APIToken for this step
#      PayloadUUID: "apollo bin.PayloadUUID" <-- take as an input to this step the named output from the "apollo bin" step
#    action_data:
#      function_name: "conditionalService" <-- the name of the conditional function to execute
#      container_name: "fileProcessor" <-- the name of the container that offers the above function
#      steps:
#        - "apollo service" <-- which step(s) we're deciding should be skipped or not

In [ ]:
## action: payload_create

#  - name: "apollo service"
#    description: "service exe with apollo shellcode"
#    action: "payload_create"
#    inputs:
#      WRAPPER_UUID: "apollo bin.PayloadUUID" <-- we're going to use the named output from "apollo bin" as an input to this step
#    depends_on:
#    - "conditional checker" <-- only execute this after this named step completed successfully
#    action_data:
#      payload_type: "service_wrapper" <-- name of the payload type we're going to use
#      description: "apollo service exe"
#      selected_os: "Windows"
#      build_parameters:
#      - name: "version"
#        value: "4.0"
#      - name: "arch"
#        value: "x64"
#      filename: "apollo_service.exe"
#      wrapped_payload: WRAPPER_UUID <-- since this is a wrapper, we provide the UUID of the payload we're going to wrap
#    outputs:
#      PayloadUUID: "uuid" <-- we can output the new UUID for this wrapped payload